In [ ]:
#First select here the benchmark_process_class_label that we will use

desired_measure = "Accuracy"
#desired_measure = "Hits-at-10"
#desired_measure = "Hits-at-1"
#desired_measure = "MRR"
#desired_measure = "AUC"


def get_figure_per_measure(desired_measure):

    
query = """
    PREFIX edam: <http://edamontology.org/>
    PREFIX obo:  <http://purl.obolibrary.org/obo/>
    PREFIX ito:  <https://identifiers.org/ito:>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>

    SELECT *
    WHERE {
        ?paper a edam:data_0971 . 
        ?paper rdfs:label ?paperTitle. 
        ?paper obo:date ?date. 

        ?benchmark_process_individual 	rdfs:seeAlso ?paper ;
                                        rdfs:label ?model_label ;
                                        a ?benchmark_process_class . # this will create a place holder for the rdfs:type results that contains the information about the individual
        
        ?benchmark_process_class rdfs:label ?benchmark_process_class_label ;
                                 rdfs:subClassOf* ito:ITO_00141 . # this limits for the NLP class


        ?performance_measure rdfs:subPropertyOf* ito:performance_measure .
        ?performance_measure rdfs:label ?metricName .



        ?performance_measure rdfs:label ?metricName .
        ?benchmark_process_individual ?performance_measure ?result .	


        #FILTER regex(?paperTitle, "Quaternion Knowledge Graph Embeddings" ). # this searches a specific match
        FILTER regex(?metricName, "^"""+ desired_measure +"""$" ). # this searches a specific match
        #FILTER regex(?benchmark_process_class_label, " - Link Prediction benchmarking$" ). # this searches a specific match
        #FILTER regex(?benchmark_process_class_label, " - Sentiment Analysis benchmarking" ). # this searches a specific match

        #WN18RR - Link Prediction benchmarking




    } ORDER by ?date



    """

#send query via API
sparql = SPARQLWrapper(endpoint)
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
result = sparql.query()

#process results as JSON
processed_results = json.load(result.response)

#Use accessory function to process results
cols = processed_results['head']['vars']

out = []
for row in processed_results['results']['bindings']:
    item = []
    for c in cols:
        item.append(row.get(c, {}).get('value'))

        out.append(item)


#this is the final df containing the results of the query
df2 = pd.DataFrame(out, columns=cols)

#now get the unique list of benchmark datasets
benchmark_process_class_label = df2["benchmark_process_class_label"].unique()

#start here the ds_count, which is actually the unique combination of 'desired_measure' and 'desired_benchmark'
ds_count = 1
#this place holder will contain the plots to be contatenated in the final plot
plt_data = []

#create final ratio_df_all for all the combinations
ratio_df_all = pd.DataFrame(columns = ["ds_count", "ds", "date", "value","gain","ratio","total"])
#fig = go.Figure()

#iterate over the benchmark list to create the ratio_df_all, which will be used to plot later
for desired_benchmark in benchmark_process_class_label:
    
    print(desired_benchmark)
    ratio_df = create_ratio(desired_measure,desired_benchmark,ds_count)
    #ratio_df = create_ratio(desired_measure,"Citeseer",ds_count)
    
    if ratio_df is None:
            print("null")  
    elif len(ratio_df.index)>0:
        print(ratio_df)  
        
        ratio_df_all = ratio_df_all.append(ratio_df, ignore_index=True)

#edit here the column with the benchmark name
ratio_df_all = ratio_df_all.replace(to_replace =' \- .*', value = '', regex = True)

#add a column representing the percentual of the max obtained value per value
ratio_df_all["percent"] = round(ratio_df_all["value"].astype(float)/max(ratio_df_all["total"].astype(float)),2)*100

#add one extra column wihth the desired measure name in order to condendate the plot
ratio_df_all["merge"]=desired_measure


# Create unique line per desired measure
fig = go.Figure(layout=go.Layout(width=800, height=400))

for N in ratio_df_all['ds_count'].unique():
    #benchmarking = re.sub(r'\-.*',"",ratio_df_all[ratio_df_all['ds_count']==N]["ds"][0])     
    #print(benchmarking)
    fig.add_trace(go.Scatter(x=ratio_df_all[ratio_df_all['ds_count']==N]["date"],
                             y=ratio_df_all[ratio_df_all['ds_count']==N]["merge"],
                    mode='lines+markers',
                    #total - local ratio
                    #marker=dict(color=ratio_df_all["total"].astype("float")-ratio_df_all["value"].astype("float"), colorscale='oranges', colorbar=dict(title='% of max. value')),
                    #total value
                    #marker=dict(color=ratio_df_all["total"].astype("float"), colorscale='oranges', colorbar=dict(title='Max. value')),
                    #total / max value
                    marker=dict(color=ratio_df_all["percent"].astype("float"), colorscale='oranges', colorbar=dict(title='Max. value')),
                    marker_size=ratio_df_all["ratio"].astype("float")*30,
                    line=dict(width=2, color='Blue' ), 
                    name="",
                    showlegend=False,
                    hovertemplate="<BR>value: "+ ratio_df_all["value"]+
                                  "<BR>total value: "+ratio_df_all["total"]+
                                  "<BR>% of max. value: "+round(((ratio_df_all["total"].astype("float")/max(ratio_df_all["total"].astype(float)))*100),2).astype("string")+"%", 
                    )
             )

#fig = go.Figure(data=go.Scatter(x=ratio_df_all["date"], y=ratio_df_all["value"]))

fig.update_layout(legend=dict(x=-.25, y=1))
fig.update_traces(marker_line_width=2, marker_line_color="black")
fig.update_layout(title=desired_measure)


fig.show()

In [16]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots


#fig = make_subplots(rows=1, cols=2)

def make_fig(i):
    fig.add_trace(go.Scatter(y=[4, i], mode="lines"))
    return fig


    
fig = go.Figure(layout=go.Layout(width=800, height=400))

x=[1,2]
for i in x:
        fig=make_fig(i)


fig.show()